In [0]:
# ======================================================================================
# NOTEBOOK DE TRANSFORMATION : FactSales
# ======================================================================================
import logging
import sys
import os
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import col, to_date, date_format

# Ajoute le dossier 'src' au chemin pour pouvoir importer notre bibliothèque
sys.path.append('../src') 
import common_utils # Importe notre bibliothèque partagée

# ======================================================================================
# 1. DÉCLARATION DES PARAMÈTRES (WIDGETS)
# ======================================================================================
dbutils.widgets.text("storage_account", "stsalesinsightcuxm0611", "Nom du compte de stockage")
dbutils.widgets.text("container", "data", "Nom du conteneur")
dbutils.widgets.text("silver_folder", "silver/sales_orders/", "Dossier source dans la couche Silver")
dbutils.widgets.text("secret_scope", "dbricks-scope-projet", "Scope unique pour les secrets du projet")
dbutils.widgets.text("adls_secret_key", "adls-access-key", "Clé du secret pour l'accès ADLS")
dbutils.widgets.text("sql_user_key", "sql-admin-user", "Clé du secret pour l'utilisateur SQL")
dbutils.widgets.text("sql_password_key", "sql-admin-password", "Clé du secret pour le mot de passe SQL")
dbutils.widgets.text("jdbc_hostname", "sqlsvr-salesinsightcuxm0611.database.windows.net", "Serveur Azure SQL DB")
dbutils.widgets.text("jdbc_database", "sqldb-salesinsight-gold", "Base de données Gold")

# ======================================================================================
# 2. FONCTION DE TRANSFORMATION SPÉCIFIQUE
# ======================================================================================

def create_fact_sales(silver_df: DataFrame, dim_product_df: DataFrame, dim_customer_df: DataFrame, dim_date_df: DataFrame, dim_status_df: DataFrame, dim_deal_size_df: DataFrame) -> DataFrame:
    """
    Crée la table de faits des ventes en joignant les données Silver avec toutes les dimensions.
    """
    logging.info("Début de la création de la table de faits 'FactSales'.")
    
    # a. Préparer le DataFrame Silver pour la jointure
    df_sales = silver_df.withColumn("DateKey", date_format(to_date(col("ORDERDATE")), "yyyyMMdd").cast("int"))

    # b. Joindre avec les dimensions pour obtenir les clés de substitution
    df_joined_prod = df_sales.join(dim_product_df, on="PRODUCTCODE", how="left")
    df_joined_cust = df_joined_prod.join(dim_customer_df, on="CUSTOMERNAME", how="left")
    df_joined_date = df_joined_cust.join(dim_date_df, on="DateKey", how="left")
    df_joined_status = df_joined_date.join(dim_status_df, df_joined_date.STATUS == dim_status_df.StatusName, how="left")
    df_joined_deal = df_joined_status.join(dim_deal_size_df, df_joined_status.DEALSIZE == dim_deal_size_df.DealSizeName, how="left")

    # c. Sélectionner les colonnes finales pour la table de faits
    fact_sales_df = df_joined_deal.select(
        col("ORDERNUMBER").alias("OrderNumber"),
        col("ProductKey"),
        col("CustomerKey"),
        col("DateKey"),
        col("StatusKey"),
        col("DealSizeKey"),
        col("QUANTITYORDERED").alias("QuantityOrdered"),
        col("PRICEEACH").alias("PriceEach"),
        col("SALES").alias("SalesAmount")
    )
    
    logging.info("Création de 'FactSales' terminée.")
    return fact_sales_df

# ======================================================================================
# 3. POINT D'ENTRÉE PRINCIPAL (MAIN)
# ======================================================================================
if __name__ == "__main__":
    
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    logging.info("===================================================")
    logging.info("DÉMARRAGE DU PIPELINE SILVER-TO-GOLD (FactSales)")
    logging.info("===================================================")
    
    try:
        # Récupération des paramètres
        storage_account = dbutils.widgets.get("storage_account").strip()
        container = dbutils.widgets.get("container").strip()
        silver_folder = dbutils.widgets.get("silver_folder").strip()
        secret_scope = dbutils.widgets.get("secret_scope").strip()
        adls_secret_key = dbutils.widgets.get("adls_secret_key").strip()
        jdbc_hostname = dbutils.widgets.get("jdbc_hostname").strip()
        jdbc_database = dbutils.widgets.get("jdbc_database").strip()
        sql_user_key = dbutils.widgets.get("sql_user_key").strip()
        sql_password_key = dbutils.widgets.get("sql_password_key").strip()

        # --- ORCHESTRATION ---
        
        # 1. Utiliser la bibliothèque pour configurer les accès
        common_utils.setup_adls_access(spark, dbutils, storage_account, secret_scope, adls_secret_key)
        jdbc_url, connection_props = common_utils.get_jdbc_connection_properties(
            dbutils, jdbc_hostname, jdbc_database, secret_scope, sql_user_key, sql_password_key
        )
        
        # 2. Lire toutes les sources de données nécessaires
        source_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{silver_folder}"
        df_silver = common_utils.read_silver_data(spark, source_path)
                

        logging.info("Lecture des tables de dimension depuis Azure SQL...")
        df_dim_product = spark.read.jdbc(url=jdbc_url, table="DimProduct", properties=connection_props)
        df_dim_customer = spark.read.jdbc(url=jdbc_url, table="DimCustomer", properties=connection_props)
        df_dim_date = spark.read.jdbc(url=jdbc_url, table="DimDate", properties=connection_props)
        df_dim_status = spark.read.jdbc(url=jdbc_url, table="DimOrderStatus", properties=connection_props)
        df_dim_deal_size = spark.read.jdbc(url=jdbc_url, table="DimDealSize", properties=connection_props)
        logging.info("Lecture des dimensions réussie.")

        # 3. Exécuter la transformation pour créer la table de faits
        fact_sales_dataframe = create_fact_sales(df_silver, df_dim_product, df_dim_customer, df_dim_date, df_dim_status, df_dim_deal_size)
        
        # 4. Utiliser la bibliothèque pour écrire la table de faits
        common_utils.write_dimension_to_gold(fact_sales_dataframe, "FactSales", jdbc_url, connection_props)
        
        logging.info("===================================================")
        logging.info("PIPELINE SILVER-TO-GOLD (FactSales) TERMINÉ AVEC SUCCÈS")
        logging.info("===================================================")

    except Exception as e:
        logging.error("Le pipeline a échoué dans le bloc principal.", exc_info=True)
        raise e
